# Model Training (Machine Learning)

We train six traditional ML classifiers on nine feature-selection datasets:

1. **Recursive Feature Elimination**
2. **Select K Best**
3. **Fisher Score Chi-Square**
4. **Extra Trees Classifier**
5. **Pearson Correlation**
6. **Mutual Information**
7. **Mutual Info Regression**
8. **Manual Uniqueness**
9. **Variance Threshold**

Each dataset trains:
- Logistic Regression  
- Gradient Boosting Classifier  
- K-Nearest Neighbours  
- Random Forest Classifier  
- Decision Tree Classifier  
- Support Vector Machine  

Metrics → `Accuracy`, `Precision`, `Recall`, `F1`  
Visuals → Confusion Matrices  
Results, Models and Figures are saved in respective folders.

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
)

DATA_DIR = Path("../data/processed/features")
PROC_DIR = Path("../data/processed")
MODEL_DIR_BASE = Path("../models")
FIG_DIR_BASE = Path("../figures")

METHODS = ["rfe","skb","fscs","etc","pc","mi","mir","mu","vt"]

## Helper Functions

These utilities:
- Train each model  
- Compute metrics (Accuracy, Precision, Recall, F1)  
- Plot and save Confusion Matrices  
- Save trained models and results

In [3]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test, model_name, method):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average="weighted", zero_division=0)
    rec = recall_score(y_test, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_test, y_pred, average="weighted", zero_division=0)

    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax)
    ax.set_title(f"{model_name} — {method.upper()}")
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    fig.tight_layout()

    fig_dir = FIG_DIR_BASE / method
    fig_dir.mkdir(parents=True, exist_ok=True)
    fig_path = fig_dir / f"{model_name.lower().replace(' ', '_')}_confusion.png"
    fig.savefig(fig_path, dpi=300, bbox_inches="tight")
    plt.close(fig)

    return {"Model": model_name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1": f1}


def save_model(model, model_name, method):
    model_dir = MODEL_DIR_BASE / method
    model_dir.mkdir(parents=True, exist_ok=True)
    path = model_dir / f"{model_name.lower().replace(' ', '_')}.pkl"
    joblib.dump(model, path)

## Model Definitions

We define six traditional ML models to train on each feature set.

In [4]:
MODELS = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Gradient Boosting": GradientBoostingClassifier(),
    "KNN": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM": SVC()
}

## Train All Six Models Across Nine Feature Sets

For each feature-selection method:
1. Load train/test CSV files  
2. Train six models  
3. Compute metrics and save results + models + confusion matrices

In [5]:
for method in METHODS:
    print(f"\n=== Training models for {method.upper()} ===")

    train_path = DATA_DIR / method / "train.csv"
    test_path = DATA_DIR / method / "test.csv"

    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    # Drop any missing rows
    train_df = train_df.dropna(subset=["DepressionEncoded"])
    test_df = test_df.dropna(subset=["DepressionEncoded"])

    X_train = train_df.drop(columns=["DepressionEncoded"])
    y_train = train_df["DepressionEncoded"].astype(int)
    X_test = test_df.drop(columns=["DepressionEncoded"])
    y_test = test_df["DepressionEncoded"].astype(int)

    results = []

    for name, model in MODELS.items():
        print(f"🧠 Training {name}")
        metrics = train_and_evaluate(model, X_train, y_train, X_test, y_test, name, method)
        results.append(metrics)
        save_model(model, name, method)

    res_df = pd.DataFrame(results)
    out_dir = PROC_DIR / method
    out_dir.mkdir(parents=True, exist_ok=True)
    res_df.to_csv(out_dir / "results_traditional_ml.csv", index=False)

    print(f"✅ Saved metrics → {out_dir / 'results_traditional_ml.csv'}")


=== Training models for RFE ===
🧠 Training Logistic Regression
🧠 Training Gradient Boosting
🧠 Training KNN
🧠 Training Random Forest
🧠 Training Decision Tree
🧠 Training SVM
✅ Saved metrics → ..\data\processed\rfe\results_traditional_ml.csv

=== Training models for SKB ===
🧠 Training Logistic Regression
🧠 Training Gradient Boosting
🧠 Training KNN
🧠 Training Random Forest
🧠 Training Decision Tree
🧠 Training SVM
✅ Saved metrics → ..\data\processed\skb\results_traditional_ml.csv

=== Training models for FSCS ===
🧠 Training Logistic Regression
🧠 Training Gradient Boosting
🧠 Training KNN
🧠 Training Random Forest
🧠 Training Decision Tree
🧠 Training SVM
✅ Saved metrics → ..\data\processed\fscs\results_traditional_ml.csv

=== Training models for ETC ===
🧠 Training Logistic Regression
🧠 Training Gradient Boosting
🧠 Training KNN
🧠 Training Random Forest
🧠 Training Decision Tree
🧠 Training SVM
✅ Saved metrics → ..\data\processed\etc\results_traditional_ml.csv

=== Training models for PC ===
🧠 Trai